In [12]:
import os
import urllib
from IPython.display import Video
import imageio.v3 as iio
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# check the video
video_name = "10515012-hd_3840_2160_24fps.mp4"
video_stem = os.path.splitext(video_name)[0]
video_path = f"../data/{video_name}"
Video(video_path, width=600)

In [ ]:
# lets download the model
model_name = "face_landmarker.task"
model_url = "https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/latest/face_landmarker.task"

os.makedirs("../models", exist_ok=True)
model_path = f"../models/{model_name}"
if not os.path.exists(model_path):
    urllib.request.urlretrieve(model_url, model_path)
    
print(f"Model {model_name} is downloaded.")

Model face_landmarker.task is downloaded.


In [ ]:
# debug function to draw the landmarks
def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

In [13]:
# prepare mediapipe settings
BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode
options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.VIDEO)

# init mediapipe
landmarker = FaceLandmarker.create_from_options(options)

# prepare video input
input = iio.imiter(video_path, plugin="pyav")
meta = iio.immeta(video_path, plugin="pyav")
fps = meta["fps"]

# prepare video output
os.makedirs("../out", exist_ok=True)
out_path = f"../out/{video_stem}_raw_detection.mp4"

# process video
with iio.imopen(out_path, "w", plugin="pyav") as output:
    output.init_video_stream("h264", fps=fps)
    
    for i, frame in enumerate(input):
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        ts = int((i / fps) * 1000)  # timestamp in milliseconds

        # Run pose detection
        detection = landmarker.detect_for_video(mp_image, timestamp_ms=ts)
        annotated = draw_landmarks_on_image(frame, detection)

        # save annotated frame to output
        output.write_frame(annotated)

landmarker.close()

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

I0000 00:00:1749415700.809462    8605 task_runner.cc:85] GPU suport is not available: INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:84) egl_initializedUnable to initialize EGL
W0000 00:00:1749415700.809941    8605 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)


In [15]:
# lets see the results
Video(out_path, width=1200)